In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install flickrapi


In [2]:
import flickrapi
import pandas as pd
#import sys



In [3]:
#set "key" and "secret" in a local configuration file named flickr_APIkey.txt
#for an example see: flickr_APIkey_example.txt
#read-in key and secret from configuration file
import os
os.chdir("/Users/KGthatsme/Analyses/FlickrFish")
file=open(file="RESULTS/flickr_APIkey.txt", mode="r") #mode r = read the file
key=file.readline()
key=key.rstrip() # Remove all trailing whitespace (e.g., \n)
secret=file.readline()
secret=secret.rstrip()

In [4]:
## Example of how to print json output of photo search
image_tag="fish"
MAX_COUNT=5
flickr = flickrapi.FlickrAPI(key, secret, format='parsed-json')
fishes = flickr.photos.search(#text=image_tag, #use text parameter instead, which searches title, description, and tags
                         #tag_mode='all',
                         tags=image_tag, 
                         #bbox='-71.791037, 41.217233, -71.095148, 42.007849', # Narraganset Bay + RI Sound
                         has_geo=1,
                         #lat=41.604168,
                         #lon=-71.320683,
                         extras='url_o, date_upload, date_taken, geo',
                         per_page=MAX_COUNT,
                         sort='relevance'
                        )
photos = fishes['photos']
from pprint import pprint
pprint(photos)

{'page': 1,
 'pages': 72250,
 'perpage': 5,
 'photo': [{'accuracy': '16',
            'context': 0,
            'datetaken': '2019-10-05 16:05:15',
            'datetakengranularity': '0',
            'datetakenunknown': '0',
            'dateupload': '1570785027',
            'farm': 66,
            'geo_is_contact': 0,
            'geo_is_family': 0,
            'geo_is_friend': 0,
            'geo_is_public': 1,
            'id': '48879645352',
            'isfamily': 0,
            'isfriend': 0,
            'ispublic': 1,
            'latitude': '52.659711',
            'longitude': '-0.691181',
            'owner': '145783492@N07',
            'place_id': '',
            'secret': '3dc6bc5708',
            'server': '65535',
            'title': "I've told you lot before - Leave the fish alone!",
            'woeid': '2635127'},
           {'accuracy': '16',
            'context': 0,
            'datetaken': '2019-09-05 10:25:09',
            'datetakengranularity': '0',
        

In [5]:
def get_urls(image_tag, MAX_COUNT):    
    flickr = flickrapi.FlickrAPI(key, secret)
    count=0
    urls=[]
    uploads=[]
    lats=[]
    longs=[]
    takens=[]
    photos = flickr.walk(#text=image_tag, #use text parameter instead, which searches title, description, and tags
                         #tag_mode='all',
                         tags=image_tag, 
                         #bbox='-71.791037, 41.217233, -71.095148, 42.007849', # Narraganset Bay + RI Sound
                         has_geo=1,
                         #lat=41.604168,
                         #lon=-71.320683,
                         extras='url_t, date_upload, date_taken, geo',
                         # use url_t instead of url_o; more search results provide access to "thumbnail" photo than the "original" photo
                         per_page=MAX_COUNT,
                         sort='relevance'
                        )
    for photo in photos:
        if count< MAX_COUNT:
            count=count+1
            print("Fetching data for image number {}".format(count))
            try:
                url=photo.get('url_t') 
                print("URL: {}".format(url))
                urls.append(url)
                upload=photo.get('dateupload')
                print("Upload: {}".format(upload))
                uploads.append(upload)
                taken=photo.get('datetaken')
                print("Taken: {}".format(taken))
                takens.append(taken)
                lat=photo.get('latitude')
                print("Latitude: {}".format(lat))
                lats.append(lat)
                long=photo.get('longitude')
                print("Longitude: {}".format(lat))
                longs.append(long)
            except:
                print("Url for image number {} could not be fetched".format(count))
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls),MAX_COUNT))
            break
    urls=pd.Series(urls)
    uploads=pd.Series(uploads)
    lats=pd.Series(lats)
    longs=pd.Series(longs)
    takens=pd.Series(takens)
    print("Writing data to the current directory")
    urls.to_csv(image_tag+"_urls.csv")
    uploads.to_csv(image_tag+"_uploads.csv")
    takens.to_csv(image_tag+"_takens.csv")
    lats.to_csv(image_tag+"_lats.csv")
    longs.to_csv(image_tag+"_longs.csv")
    print("Done!!!")



In [6]:
# change wd to RESULTS folder
os.chdir("/Users/KGthatsme/Analyses/FlickrFish/RESULTS")

In [7]:
image_tag="fish"
MAX_COUNT=100
get_urls(image_tag=image_tag, MAX_COUNT=MAX_COUNT)

Fetching data for image number 1
URL: https://live.staticflickr.com/65535/48879645352_3dc6bc5708_t.jpg
Upload: 1570785027
Taken: 2019-10-05 16:05:15
Latitude: 52.659711
Longitude: 52.659711
Fetching data for image number 2
URL: https://live.staticflickr.com/65535/48857453767_bd1c34522a_t.jpg
Upload: 1570422668
Taken: 2019-09-05 10:25:09
Latitude: -17.263656
Longitude: -17.263656
Fetching data for image number 3
URL: https://live.staticflickr.com/65535/48847015638_6f92a98614_t.jpg
Upload: 1570433105
Taken: 2014-10-27 13:37:10
Latitude: 51.435372
Longitude: 51.435372
Fetching data for image number 4
URL: https://live.staticflickr.com/65535/48840571261_9b2539c694_t.jpg
Upload: 1570168896
Taken: 2019-08-10 15:21:35
Latitude: 54.177763
Longitude: 54.177763
Fetching data for image number 5
URL: https://live.staticflickr.com/65535/48824095221_c418e51440_t.jpg
Upload: 1569892921
Taken: 2019-08-03 15:44:34
Latitude: 12.059550
Longitude: 12.059550
Fetching data for image number 6
URL: https://li

Done fetching urls, fetched 100 urls out of 100
Writing data to the current directory
Done!!!


In [8]:
## Only need the following code if running this from Terminal
#def main(): 
#    tag=sys.argv[1]
#    MAX_COUNT=int(sys.argv[2])
#    get_urls(tag,MAX_COUNT)


#if __name__=='__main__': 
#    main() 
# if running the module as the main program (e.g., in terminal: python getFishDat.py): execute function (main)
# if module is imported by another program, _name_ is assigned to the module name "getFishDat": skip function (main)

import csv
import requests
import os
import sys
import time
def put_images(image_tag):
    urls=[]
    FILE_NAME=image_tag+"_urls.csv"
    with open(FILE_NAME,newline="") as csvfile:
        doc=csv.reader(csvfile,delimiter=",")
        for row in doc:
            if row[1].startswith("https"):
                urls.append(row[1])
    if not os.path.isdir(os.path.join(os.getcwd(),FILE_NAME.split("_")[0])):
        os.mkdir(FILE_NAME.split("_")[0])
    t0=time.time()
    for url in enumerate(urls):
        print("Starting download {} of ".format(url[0]+1),len(urls))
        try:
            resp=requests.get(url[1],stream=True)
            path_to_write=os.path.join(os.getcwd(),FILE_NAME.split("_")[0],url[1].split("/")[-1])
            outfile=open(path_to_write,'wb')
            outfile.write(resp.content)
            outfile.close()
            print("Done downloading {} of {}".format(url[0]+1,len(urls)))
        except:
            print("Failed to download url number {}".format(url[0]))
    t1=time.time()
    print("Done with download, job took {} seconds".format(t1-t0))





In [ ]:
put_images(image_tag=image_tag)

Starting download 1 of  100
Done downloading 1 of 100
Starting download 2 of  100
Done downloading 2 of 100
Starting download 3 of  100
Done downloading 3 of 100
Starting download 4 of  100
Done downloading 4 of 100
Starting download 5 of  100
Done downloading 5 of 100
Starting download 6 of  100
Done downloading 6 of 100
Starting download 7 of  100
Done downloading 7 of 100
Starting download 8 of  100
Done downloading 8 of 100
Starting download 9 of  100
Done downloading 9 of 100
Starting download 10 of  100
